In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-13 21:25:30--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.09s   

2022-08-13 21:25:30 (10.1 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("illegal_postings_311").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"),sep=r'\t', header=True, inferSchema=True, timestampFormat="yyyy/MM/dd HH:mm:ss")\
                                  .select("marketplace",      
"customer_id",     
"review_id",        
"product_id",       
                 
"product_parent",   
"product_title",    
"product_category", 
                 
"star_rating",      
"helpful_votes",    
"total_votes",      
"vine",             
"verified_purchase",
"review_headline",
"review_body",      
"review_date"    )

# Show DataFrame
col = len(df.columns)

col

df = df.drop("marketplace")

df.show()

+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|          5|            0|          1|   N|                Y|     

In [6]:
df.count()

104975

In [7]:
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

review_id_df = df.select(["review_id","customer_id","product_id","product_parent","review_date"])
review_id_df1 = review_id_df.dropna()

product_df = df.select(["product_id","product_title"])

product_df1 = product_df.dropna()

product_df1.count()




unique_df1 = df.select(["product_id",'product_title'])

unique_df2 = unique_df1.drop_duplicates()

# unique_df1.show()

unique_df1.count()
# from pyspark.sql.functions import col,isnan, when, count
# from pyspark.sql import SparkSession



# import pyspark.sql.functions as f
# unique_df2.join(
#     unique_df2.groupBy(unique_df2.columns).agg((f.count("*")>1).cast("int").alias("Duplicate_indicator")),
#     on=unique_df2.columns,
#     how="inner"
# ).show()



104975

In [8]:
g_unique = unique_df2.groupby("product_id").count()

g_unique1 = unique_df2.groupby("product_title").count()
g_unique.count()

25801

In [9]:
clean_unique = unique_df2.drop_duplicates()

clean_df1 = clean_unique.distinct()

clean_df2 = clean_df1.dropna()

clean_df2.count()

25801

In [10]:
from pyspark.sql.functions import col, countDistinct

unique_df3 = unique_df2.agg(*(countDistinct(col(c)).alias(c) for c in unique_df2.columns))

unique_df3.select("product_title").distinct().show()

+-------------+
|product_title|
+-------------+
|        24770|
+-------------+



In [15]:
import getpass
password = getpass.getpass('Enter database password: ')
# password = input("write password: ")
mode = "append"
jdbc_url="jdbc:postgresql://review.c5dan5rmmbko.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password: ··········


In [12]:
# unique_df2.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)